In [1]:
import pandas as pd
import numpy as np
import os

### Read Hamal data

In [2]:
df_hamal = pd.read_excel('hamal.xlsx')
df_hamal.head()

,StatZone,Street,HouseNum,Year,Month,Day,DayWeek,ResNum,SenWid,SenCitWid,...,HouUsageEss,T_HouUsageEss,HouOwnAge,HouOwnFamStat,RentOrOwn,HouUsage,ArnonaDisc,TotHouArea,MevuneArea,Karka
0,611,דמה,דמה,2019.0,12.0,NaN,NaN,4228.0,42.0,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,612,דמה,דמה,2019.0,12.0,NaN,NaN,2042.0,23.0,108.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,613,דמה,דמה,2019.0,12.0,NaN,NaN,1974.0,16.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621,דמה,דמה,2019.0,12.0,NaN,NaN,4337.0,25.0,262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,622,דמה,דמה,2019.0,12.0,NaN,NaN,2960.0,26.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### The data is sparse, it is a join of many small data frames. We will split it to dmall dfs and save them for easier use.

In [3]:
df_hamal_columns = list(df_hamal.columns)
base_columns = df_hamal_columns[:7]
small_df_division_dict = {"blue"        : df_hamal_columns[7:71],
                          "red"         : df_hamal_columns[71:81],
                          "yellow"      : df_hamal_columns[81:82] + df_hamal_columns[96:98] ,
                          "grey"        : df_hamal_columns[82:86],
                          "brown"       : df_hamal_columns[86:88],
                          "light_blue"  : df_hamal_columns[88:89],
                          "pink"        : df_hamal_columns[89:90],
                          "green"       : df_hamal_columns[90:93],
                          "orange"      : df_hamal_columns[93:96],
                          "purple"      : df_hamal_columns[98:103],
                          "dark_blue"   : df_hamal_columns[103:105],
                          "shiny_green" : df_hamal_columns[105:]}

# Let's open a dir for all the small dfs
directory = 'hamal_data_splitted'
if not os.path.exists(directory):
    os.makedirs(directory)

for df_name, df_columns in small_df_division_dict.items():
    df_hamal_temp = df_hamal[base_columns + df_columns].copy().dropna(subset=df_columns)
    df_hamal_temp.to_csv(directory + '/' + df_name + '.csv', index=False,encoding = 'utf-8-sig')

### We want a mapping of statictical areas to street name

In [4]:
df_statistical_areas_to_streets = df_hamal[['Street','StatZone']].drop_duplicates()
df_statistical_areas_to_streets = df_statistical_areas_to_streets[df_statistical_areas_to_streets['Street'] != 'דמה'].reset_index().drop(['index'],axis=1)
df_statistical_areas_to_streets.to_csv('stats_zones_to_strees.csv',  index=False,encoding = 'utf-8-sig')

### Let's read the crime related data from the Hamal - red & brown df

In [5]:
df_red = pd.read_csv( directory + '/' + 'red.csv',encoding = 'utf-8-sig')
df_brown = pd.read_csv(directory + '/'+ 'brown.csv',encoding = 'utf-8-sig')

### We want to understand if the data in the red df is a sum of the crimes in 2012 in the brown df

In [6]:
df_brown_2012 = df_brown[df_brown['Year'] == 2012]
number_of_crimes_in_2012_crown = len(df_brown_2012)
df_red['sum_row'] = df_red.apply(lambda row: row[small_df_division_dict['red']].sum(), axis=1)
print(number_of_crimes_in_2012_crown, sum(df_red['sum_row']))

4988 4314.0


In [7]:
df_brown_crime_types = list(df_brown['CrimeType'].unique())
print(small_df_division_dict['red'])
print(df_brown_crime_types)

['TotPropCrime', 'TotCrimCrime', 'TotBusiBurg', 'TotApartBurg', 'CarTheft', 'TotPropCrime.1', 'Thefts', 'BodyAssaults', 'SexualAssaults', 'Robbery']
['עבירות כלפי הרכוש', 'עבירות מרמה', 'עבירות סדר ציבורי', 'עבירות כלפי המוסר', 'שאר עבירות', 'עבירות נגד גוף', 'עבירות רשוי', 'עבירות בטחון', 'עבירות מין', 'עבירות תנועה', 'עבירות נגד אדם', 'עבירות כלכליות']


* We can see that the type of crime in the 2 df disagree.. Therfor we will use only the brown df which contains more data & more years

* We will create a mapping of crimes in each year & stats zone, crime_type (avg, sum). 

In [8]:
df_crimes_in_stats_zones = df_brown.groupby(['StatZone','Year','CrimeType']).size().reset_index()
df_crimes_in_stats_zones = df_crimes_in_stats_zones.rename({0: 'NumCrimes'},axis=1)

In [9]:
# df_crimes_in_stats_zones.to_csv('crimes_ins_stats_zones.csv',  index=False,encoding = 'utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'crimes_ins_stats_zones.csv'

In [10]:
df_crimes_in_stats_zones.head()

,StatZone,Year,CrimeType,NumCrimes
0,611,2010.0,עבירות כלפי המוסר,2
1,611,2010.0,עבירות כלפי הרכוש,52
2,611,2010.0,עבירות מין,2
3,611,2010.0,עבירות מרמה,3
4,611,2010.0,עבירות נגד גוף,26


In [11]:
df_crimes_in_stats_zones_count_year = df_crimes_in_stats_zones.groupby(['Year', 'StatZone']).size().reset_index()

In [12]:
df_crimes_in_stats_zones_count_year

,Year,StatZone,0
0,2010.0,611,8
1,2010.0,612,5
2,2010.0,613,8
3,2010.0,621,10
4,2010.0,622,9
5,2010.0,623,7
6,2010.0,631,10
7,2010.0,632,9
8,2010.0,633,10
9,2010.0,634,9


* We can see that if we look in granularity for 'year', 'statZone' , not all the Crime Types occured. So we will use weights for the different CrimeTypes and probably for the years as well (as later is the year - the weight should increase?)

In [15]:
df_crimes_in_stats_zones_sum = df_crimes_in_stats_zones.groupby(['StatZone', 'CrimeType']).agg({'NumCrimes' : 'sum'}).reset_index()

In [17]:
types_of_crimes = { 'sex_crimes' : ['עבירות מין'], 'violent_crimes' : ['עבירות נגד אדם', 'עבירות בטחון'], 'other_crimes' : ['עבירות כלפי הרכוש', 'עבירות מרמה', 'עבירות סדר ציבורי', 'עבירות כלפי המוסר', 'שאר עבירות', 'עבירות נגד גוף', 'עבירות רשוי', 'עבירות תנועה', 'עבירות כלכליות']}
df_crimes_in_stats_zones_sum['CrimeTypeModifies'] = df_crimes_in_stats_zones_sum.apply(lambda row: 'sex_crimes' if row['CrimeType'] in types_of_crimes['sex_crimes'] else 'violent_crimes' if row['CrimeType'] in types_of_crimes['violent_crimes'] else 'other_crimes', axis=1  )

In [19]:
df_crimes_in_stats_zones_sum_grouped = df_crimes_in_stats_zones_sum[['StatZone','CrimeTypeModifies','NumCrimes']].groupby(['StatZone','CrimeTypeModifies']).agg({'NumCrimes' : 'sum'}).reset_index()

In [31]:
weights_of_crimes_dict = {'sex_crimes' : 0.45, 'violent_crimes' : 0.45,'other_crimes' : 0.1 }
df_crimes_in_stats_zones_sum_grouped['weight'] = df_crimes_in_stats_zones_sum_grouped.apply(lambda row: row['NumCrimes'] * weights_of_crimes_dict[row['CrimeTypeModifies']], axis=1 )

In [33]:
df_crimes_in_stats_zones_sum_weighted = df_crimes_in_stats_zones_sum_grouped.groupby(['StatZone']).agg({'weight' : 'sum'}).reset_index()

In [34]:
df_crimes_in_stats_zones_sum_weighted

,StatZone,weight
0,611,95.10
1,612,96.20
2,613,159.95
3,621,205.45
4,622,127.45
5,623,67.45
6,631,247.60
7,632,316.10
8,633,583.40
9,634,556.75


In [38]:
# # Using sklearn
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler() 
# arr_scaled = scaler.fit_transform(df_crimes_in_stats_zones_sum_weighted[['weight']]) 
df_crimes_in_stats_zones_sum_weighted['normalized_weight'] = (df_crimes_in_stats_zones_sum_weighted['weight']-df_crimes_in_stats_zones_sum_weighted['weight'].min())/(df_crimes_in_stats_zones_sum_weighted['weight'].max()-df_crimes_in_stats_zones_sum_weighted['weight'].min())

In [39]:
df_crimes_in_stats_zones_sum_weighted

,StatZone,weight,normalized_weight
0,611,95.10,0.141224
1,612,96.20,0.143159
2,613,159.95,0.255276
3,621,205.45,0.335297
4,622,127.45,0.198118
5,623,67.45,0.092596
6,631,247.60,0.409427
7,632,316.10,0.529898
8,633,583.40,1.000000
9,634,556.75,0.953130


In [40]:
df_crimes_in_stats_zones_sum_weighted.to_csv('weighted_crimes_in_stats_zones.csv', index=False)

In [ ]:
def apply_weightes_from_params(stairs_weight, 
                               is_parks,
                               security_weight,
                               is_sport,
                               km_sport,
                               is_public_gym)
    
    if is_sport:
        df_park_and_sports = pd.read_csv()
    for all edjes in the graph:
        if type == stairs:
            edje wieght = wieght*stairs_weight
        elif security_weight > 1:
            edje wieght = wieght + ztats_zone wieght * (security_weight-1)
        
        for two nodes in edge
            node1 = df_park_and_sports[df_park_and_sports['node'] == node_edge]
            

        


        
        